In [1]:

import os
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse



In [2]:

DB_USER = "postgres"
DB_PASSWORD = urllib.parse.quote_plus("Mahivalli@24")  
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "stockhost"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")



In [ ]:

master_df = pd.read_csv("master_csv.csv")
output_folder = "ticker_wise_data"
os.makedirs(output_folder, exist_ok=False)


In [13]:


ticker_groups = {ticker: df for ticker, df in master_df.groupby("Ticker")}

for ticker, df in ticker_groups.items():
    file_path = os.path.join(output_folder, f"{ticker}.csv")
    df.to_csv(file_path, index=False)
    print(f"📁 Saved {ticker} data to {file_path}")


📁 Saved ADANIENT data to ticker_wise_data\ADANIENT.csv
📁 Saved ADANIPORTS data to ticker_wise_data\ADANIPORTS.csv
📁 Saved APOLLOHOSP data to ticker_wise_data\APOLLOHOSP.csv
📁 Saved ASIANPAINT data to ticker_wise_data\ASIANPAINT.csv
📁 Saved AXISBANK data to ticker_wise_data\AXISBANK.csv
📁 Saved BAJAJ-AUTO data to ticker_wise_data\BAJAJ-AUTO.csv
📁 Saved BAJAJFINSV data to ticker_wise_data\BAJAJFINSV.csv
📁 Saved BAJFINANCE data to ticker_wise_data\BAJFINANCE.csv
📁 Saved BEL data to ticker_wise_data\BEL.csv
📁 Saved BHARTIARTL data to ticker_wise_data\BHARTIARTL.csv
📁 Saved BPCL data to ticker_wise_data\BPCL.csv
📁 Saved BRITANNIA data to ticker_wise_data\BRITANNIA.csv
📁 Saved CIPLA data to ticker_wise_data\CIPLA.csv
📁 Saved COALINDIA data to ticker_wise_data\COALINDIA.csv
📁 Saved DRREDDY data to ticker_wise_data\DRREDDY.csv
📁 Saved EICHERMOT data to ticker_wise_data\EICHERMOT.csv
📁 Saved GRASIM data to ticker_wise_data\GRASIM.csv
📁 Saved HCLTECH data to ticker_wise_data\HCLTECH.csv
📁 Saved 

In [6]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import urllib.parse
import re

# Database connection details
DB_USER = "postgres"
DB_PASSWORD = urllib.parse.quote_plus("Mahivalli@24")
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "stockhost"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Function to sanitize ticker names
def sanitize_ticker(ticker: str) -> str:
    return re.sub(r'[^a-zA-Z0-9]', '_', ticker).lower()

# Input folder containing CSV files
input_folder = "ticker_wise_data"
performance_summary = []
all_data = []

# Process each CSV file
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        ticker = file.replace(".csv", "")
        file_path = os.path.join(input_folder, file)
        try:
            df = pd.read_csv(file_path)
            df["date"] = pd.to_datetime(df["date"])
            df = df.sort_values("date")

            # Daily return calculation
            df["daily_return"] = df["close"].pct_change()

            # ✅ Daily cumulative return progression (for line chart)
            P_start = df["close"].iloc[0]
            df["cumulative_return"] = ((df["close"] - P_start) / P_start) * 100

            # ✅ Final cumulative return (scalar for summary table)
            final_cum_return = df["cumulative_return"].iloc[-1]

            performance_summary.append({
                "Ticker": ticker,
                "FinalCumulativeReturn": final_cum_return
            })

            df["Ticker"] = ticker
            all_data.append(df)

        except Exception as e:
            print(f"Error processing {file_path}: {e}")

# Combine all data
if all_data:
    all_df = pd.concat(all_data, ignore_index=True)
    all_df["Ticker"] = all_df["Ticker"].astype(str)

# Volatility analysis
if not all_df.empty:
    volatility_df = (
        all_df.groupby("Ticker")["daily_return"]
        .std()
        .reset_index(name="Volatility")
        .sort_values("Volatility", ascending=False)
        .head(10)
    )
    volatility_df.to_sql("top10_volatility", engine, if_exists="replace", index=False)
    print("✅ Volatility analysis stored in DB (top10_volatility)")
else:
    print("⚠️ No data available for volatility analysis")

# Performance summary analysis
perf_df = pd.DataFrame(performance_summary)
if not perf_df.empty:
    top5_perf = perf_df.sort_values("FinalCumulativeReturn", ascending=False).head(5)
    top5_perf.to_sql("top5_cumulative_performance", engine, if_exists="replace", index=False)
    print("✅ Cumulative return analysis stored in DB (top5_cumulative_performance)")
else:
    print("⚠️ No cumulative performance data generated")

✅ Volatility analysis stored in DB (top10_volatility)
✅ Cumulative return analysis stored in DB (top5_cumulative_performance)


In [ ]:
import pandas as pd


master_file = "master_csv.csv"   
df = pd.read_csv(master_file)

if "Ticker" not in df.columns:
    raise ValueError("Master CSV must contain a 'Ticker' column")

unique_tickers = df["Ticker"].unique()

print("✅ Unique Tickers Found:")
print(unique_tickers)

pd.DataFrame(unique_tickers, columns=["UniqueTicker"]).to_csv("unique_tickers.csv", index=False)
print("Saved unique tickers to unique_tickers.csv")

✅ Unique Tickers Found:
['SBIN' 'BAJFINANCE' 'TITAN' 'ITC' 'TCS' 'LT' 'TATACONSUM' 'RELIANCE'
 'HCLTECH' 'JSWSTEEL' 'ULTRACEMCO' 'POWERGRID' 'INFY' 'TRENT' 'BHARTIARTL'
 'TATAMOTORS' 'WIPRO' 'TECHM' 'NTPC' 'HINDUNILVR' 'APOLLOHOSP' 'M&M'
 'GRASIM' 'ICICIBANK' 'ADANIENT' 'ADANIPORTS' 'BEL' 'BAJAJFINSV'
 'EICHERMOT' 'COALINDIA' 'MARUTI' 'INDUSINDBK' 'ASIANPAINT' 'TATASTEEL'
 'HDFCLIFE' 'DRREDDY' 'SUNPHARMA' 'KOTAKBANK' 'SHRIRAMFIN' 'NESTLEIND'
 'ONGC' 'CIPLA' 'BPCL' 'BRITANNIA' 'SBILIFE' 'HINDALCO' 'HEROMOTOCO'
 'AXISBANK' 'HDFCBANK' 'BAJAJ-AUTO']
✅ Saved unique tickers to unique_tickers.csv


In [7]:
import pandas as pd
tickers = [
    'SBIN','BAJFINANCE','TITAN','ITC','TCS','LT','TATACONSUM','RELIANCE',
    'HCLTECH','JSWSTEEL','ULTRACEMCO','POWERGRID','INFY','TRENT','BHARTIARTL',
    'TATAMOTORS','WIPRO','TECHM','NTPC','HINDUNILVR','APOLLOHOSP','M&M',
    'GRASIM','ICICIBANK','ADANIENT','ADANIPORTS','BEL','BAJAJFINSV',
    'EICHERMOT','COALINDIA','MARUTI','INDUSINDBK','ASIANPAINT','TATASTEEL',
    'HDFCLIFE','DRREDDY','SUNPHARMA','KOTAKBANK','SHRIRAMFIN','NESTLEIND',
    'ONGC','CIPLA','BPCL','BRITANNIA','SBILIFE','HINDALCO','HEROMOTOCO',
    'AXISBANK','HDFCBANK','BAJAJ-AUTO'
]

sector_map = {
    # Banking & Financials
    'SBIN':'Banking & Financials','BAJFINANCE':'Banking & Financials','ICICIBANK':'Banking & Financials',
    'AXISBANK':'Banking & Financials','HDFCBANK':'Banking & Financials','INDUSINDBK':'Banking & Financials',
    'KOTAKBANK':'Banking & Financials','SHRIRAMFIN':'Banking & Financials','BAJAJFINSV':'Banking & Financials',
    'SBILIFE':'Banking & Financials','HDFCLIFE':'Banking & Financials',

    # IT
    'TCS':'IT','INFY':'IT','WIPRO':'IT','HCLTECH':'IT','TECHM':'IT',

    # Industrials & Infrastructure
    'LT':'Industrials','POWERGRID':'Industrials','NTPC':'Industrials','JSWSTEEL':'Industrials',
    'TATASTEEL':'Industrials','HINDALCO':'Industrials','COALINDIA':'Industrials','ULTRACEMCO':'Industrials','GRASIM':'Industrials',

    # Automobiles
    'TATAMOTORS':'Automobile','M&M':'Automobile','EICHERMOT':'Automobile','HEROMOTOCO':'Automobile',
    'BAJAJ-AUTO':'Automobile','MARUTI':'Automobile',

    # Energy & Oil & Gas
    'RELIANCE':'Energy','ONGC':'Energy','BPCL':'Energy',

    # FMCG & Consumer Goods
    'ITC':'FMCG','HINDUNILVR':'FMCG','NESTLEIND':'FMCG','BRITANNIA':'FMCG','ASIANPAINT':'FMCG',
    'TATACONSUM':'FMCG','TRENT':'FMCG','TITAN':'FMCG',

    # Healthcare & Pharma
    'DRREDDY':'Healthcare','SUNPHARMA':'Healthcare','CIPLA':'Healthcare','APOLLOHOSP':'Healthcare',

    # Telecom
    'BHARTIARTL':'Telecom',

    # Ports & Logistics
    'ADANIPORTS':'Ports & Logistics','ADANIENT':'Ports & Logistics',

    # Defence & Electronics
    'BEL':'Defence & Electronics'
}


sector_df = pd.DataFrame({"Ticker": tickers})
sector_df["Sector"] = sector_df["Ticker"].map(sector_map)


sector_df.to_csv("sector_wise_tickers.csv", index=False)


print("✅ Sector-wise classification saved to sector_wise_tickers.csv")
display(sector_df)

✅ Sector-wise classification saved to sector_wise_tickers.csv


,Ticker,Sector
0,SBIN,Banking & Financials
1,BAJFINANCE,Banking & Financials
2,TITAN,FMCG
3,ITC,FMCG
4,TCS,IT
5,LT,Industrials
6,TATACONSUM,FMCG
7,RELIANCE,Energy
8,HCLTECH,IT
9,JSWSTEEL,Industrials


In [8]:
import os
import pandas as pd

master_file = "master_csv.csv"   
df = pd.read_csv(master_file)

sector_map = {
    # Banking & Financials
    'SBIN':'Banking','BAJFINANCE':'Banking','ICICIBANK':'Banking','AXISBANK':'Banking',
    'HDFCBANK':'Banking','INDUSINDBK':'Banking','KOTAKBANK':'Banking','SHRIRAMFIN':'Banking',
    'BAJAJFINSV':'Banking','SBILIFE':'Banking','HDFCLIFE':'Banking',

    # IT
    'TCS':'IT','INFY':'IT','WIPRO':'IT','HCLTECH':'IT','TECHM':'IT',

    # Industrials
    'LT':'Industrials','POWERGRID':'Industrials','NTPC':'Industrials','JSWSTEEL':'Industrials',
    'TATASTEEL':'Industrials','HINDALCO':'Industrials','COALINDIA':'Industrials',
    'ULTRACEMCO':'Industrials','GRASIM':'Industrials',

    # Automobiles
    'TATAMOTORS':'Automobile','M&M':'Automobile','EICHERMOT':'Automobile',
    'HEROMOTOCO':'Automobile','BAJAJ-AUTO':'Automobile','MARUTI':'Automobile',

    # Energy
    'RELIANCE':'Energy','ONGC':'Energy','BPCL':'Energy',

    # FMCG & Consumer Goods
    'ITC':'FMCG','HINDUNILVR':'FMCG','NESTLEIND':'FMCG','BRITANNIA':'FMCG',
    'ASIANPAINT':'FMCG','TATACONSUM':'FMCG','TRENT':'FMCG','TITAN':'FMCG',

    # Healthcare
    'DRREDDY':'Healthcare','SUNPHARMA':'Healthcare','CIPLA':'Healthcare','APOLLOHOSP':'Healthcare',

    # Telecom
    'BHARTIARTL':'Telecom',

    # Ports & Logistics
    'ADANIPORTS':'Ports & Logistics','ADANIENT':'Ports & Logistics',

    # Defence
    'BEL':'Defence'
}
df["Sector"] = df["Ticker"].map(sector_map)

base_folder = "sector_wise_data"
os.makedirs(base_folder, exist_ok=True)

for sector, group in df.groupby("Sector"):
    sector_folder = os.path.join(base_folder, sector)
    os.makedirs(sector_folder, exist_ok=True)
    
    sector_file = os.path.join(sector_folder, f"{sector.lower()}_data.csv")
    group.to_csv(sector_file, index=False)
    
    print(f"✅ Saved {sector} data to {sector_file}")


sector_summary = df.groupby("Sector")["Ticker"].nunique().reset_index(name="UniqueTickerCount")
print("\n Sector Summary:\n", sector_summary)

✅ Saved Automobile data to sector_wise_data\Automobile\automobile_data.csv
✅ Saved Banking data to sector_wise_data\Banking\banking_data.csv
✅ Saved Defence data to sector_wise_data\Defence\defence_data.csv
✅ Saved Energy data to sector_wise_data\Energy\energy_data.csv
✅ Saved FMCG data to sector_wise_data\FMCG\fmcg_data.csv
✅ Saved Healthcare data to sector_wise_data\Healthcare\healthcare_data.csv
✅ Saved IT data to sector_wise_data\IT\it_data.csv
✅ Saved Industrials data to sector_wise_data\Industrials\industrials_data.csv
✅ Saved Ports & Logistics data to sector_wise_data\Ports & Logistics\ports & logistics_data.csv
✅ Saved Telecom data to sector_wise_data\Telecom\telecom_data.csv

 Sector Summary:
               Sector  UniqueTickerCount
0         Automobile                  6
1            Banking                 11
2            Defence                  1
3             Energy                  3
4               FMCG                  8
5         Healthcare                  4
6       

In [9]:
import pandas as pd


master_file = "master_csv.csv"   
df = pd.read_csv(master_file)

sector_map = {
    'SBIN':'Banking','BAJFINANCE':'Banking','ICICIBANK':'Banking','AXISBANK':'Banking',
    'HDFCBANK':'Banking','INDUSINDBK':'Banking','KOTAKBANK':'Banking','SHRIRAMFIN':'Banking',
    'BAJAJFINSV':'Banking','SBILIFE':'Banking','HDFCLIFE':'Banking',
    'TCS':'IT','INFY':'IT','WIPRO':'IT','HCLTECH':'IT','TECHM':'IT',
    'LT':'Industrials','POWERGRID':'Industrials','NTPC':'Industrials','JSWSTEEL':'Industrials',
    'TATASTEEL':'Industrials','HINDALCO':'Industrials','COALINDIA':'Industrials',
    'ULTRACEMCO':'Industrials','GRASIM':'Industrials',
    'TATAMOTORS':'Automobile','M&M':'Automobile','EICHERMOT':'Automobile',
    'HEROMOTOCO':'Automobile','BAJAJ-AUTO':'Automobile','MARUTI':'Automobile',
    'RELIANCE':'Energy','ONGC':'Energy','BPCL':'Energy',
    'ITC':'FMCG','HINDUNILVR':'FMCG','NESTLEIND':'FMCG','BRITANNIA':'FMCG',
    'ASIANPAINT':'FMCG','TATACONSUM':'FMCG','TRENT':'FMCG','TITAN':'FMCG',
    'DRREDDY':'Healthcare','SUNPHARMA':'Healthcare','CIPLA':'Healthcare','APOLLOHOSP':'Healthcare',
    'BHARTIARTL':'Telecom',
    'ADANIPORTS':'Ports & Logistics','ADANIENT':'Ports & Logistics',
    'BEL':'Defence'
}

df["Sector"] = df["Ticker"].map(sector_map)

updated_file = "master_csv_with_sector.csv"
df.to_csv(updated_file, index=False)

print(f" Master CSV updated with Sector column and saved as {updated_file}")
display(df.head())

 Master CSV updated with Sector column and saved as master_csv_with_sector.csv


,Ticker,close,date,high,low,month,open,volume,Sector
0,SBIN,602.95,2023-10-03 05:30:00,604.9,589.60,2023-10,596.6,15322196,Banking
1,BAJFINANCE,7967.60,2023-10-03 05:30:00,7975.5,7755.00,2023-10,7780.8,944555,Banking
2,TITAN,3196.25,2023-10-03 05:30:00,3212.5,3114.40,2023-10,3148.8,1007308,FMCG
3,ITC,439.75,2023-10-03 05:30:00,442.9,439.25,2023-10,441.0,7481883,FMCG
4,TCS,3513.85,2023-10-03 05:30:00,3534.2,3480.10,2023-10,3534.2,1948148,IT


In [10]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse

DB_USER = "postgres"
DB_PASSWORD = urllib.parse.quote_plus("Mahivalli@24")
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "stockhost"
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

master_file = "master_csv.csv"   
df = pd.read_csv(master_file)

sector_map = {
    'SBIN':'Banking','BAJFINANCE':'Banking','ICICIBANK':'Banking','AXISBANK':'Banking',
    'HDFCBANK':'Banking','INDUSINDBK':'Banking','KOTAKBANK':'Banking','SHRIRAMFIN':'Banking',
    'BAJAJFINSV':'Banking','SBILIFE':'Banking','HDFCLIFE':'Banking',
    'TCS':'IT','INFY':'IT','WIPRO':'IT','HCLTECH':'IT','TECHM':'IT',
    'LT':'Industrials','POWERGRID':'Industrials','NTPC':'Industrials','JSWSTEEL':'Industrials',
    'TATASTEEL':'Industrials','HINDALCO':'Industrials','COALINDIA':'Industrials',
    'ULTRACEMCO':'Industrials','GRASIM':'Industrials',
    'TATAMOTORS':'Automobile','M&M':'Automobile','EICHERMOT':'Automobile',
    'HEROMOTOCO':'Automobile','BAJAJ-AUTO':'Automobile','MARUTI':'Automobile',
    'RELIANCE':'Energy','ONGC':'Energy','BPCL':'Energy',
    'ITC':'FMCG','HINDUNILVR':'FMCG','NESTLEIND':'FMCG','BRITANNIA':'FMCG',
    'ASIANPAINT':'FMCG','TATACONSUM':'FMCG','TRENT':'FMCG','TITAN':'FMCG',
    'DRREDDY':'Healthcare','SUNPHARMA':'Healthcare','CIPLA':'Healthcare','APOLLOHOSP':'Healthcare',
    'BHARTIARTL':'Telecom',
    'ADANIPORTS':'Ports & Logistics','ADANIENT':'Ports & Logistics',
    'BEL':'Defence'
}

df["Sector"] = df["Ticker"].map(sector_map)

df["date"] = pd.to_datetime(df["date"])
df["Year"] = df["date"].dt.year

yearly_returns = df.groupby(["Ticker", "Sector", "Year"])["close"].apply(
    lambda x: x.iloc[-1] / x.iloc[0] - 1 if len(x) > 1 else None
).reset_index(name="YearlyReturn")

sector_perf = yearly_returns.groupby(["Sector", "Year"])["YearlyReturn"].mean().reset_index()

sector_perf.to_sql("sector_performance", engine, if_exists="replace", index=False)
print("✅ Sector-wise performance stored in DB (sector_performance)")

sector_perf.head()

✅ Sector-wise performance stored in DB (sector_performance)


,Sector,Year,YearlyReturn
0,Automobile,2023,0.223617
1,Automobile,2024,0.276492
2,Banking,2023,0.066626
3,Banking,2024,0.047865
4,Defence,2023,0.323276


In [12]:
import pandas as pd
df = pd.read_csv("master_csv.csv")
pivot_df = df.pivot(index="date", columns="Ticker", values="close")
returns_df = pivot_df.pct_change().dropna()
corr_matrix = returns_df.corr()
print(" Correlation Matrix of Stock Returns:")
print(corr_matrix)
corr_matrix_reset = corr_matrix.reset_index()
#corr_matrix_reset.to_sql("correlation_matrix", engine, if_exists="replace", index=False)
print("✅ Correlation matrix stored in DB (correlation_matrix)")



 Correlation Matrix of Stock Returns:
Ticker      ADANIENT  ADANIPORTS  APOLLOHOSP  ASIANPAINT  AXISBANK  \
Ticker                                                               
ADANIENT    1.000000    0.874233    0.136501    0.291583  0.297736   
ADANIPORTS  0.874233    1.000000    0.168902    0.274045  0.380514   
APOLLOHOSP  0.136501    0.168902    1.000000    0.258055  0.179381   
ASIANPAINT  0.291583    0.274045    0.258055    1.000000  0.126055   
AXISBANK    0.297736    0.380514    0.179381    0.126055  1.000000   
BAJAJ-AUTO  0.201871    0.204456    0.223900    0.167650  0.198477   
BAJAJFINSV  0.356547    0.398041    0.249392    0.248090  0.393683   
BAJFINANCE  0.336031    0.398769    0.223142    0.296922  0.354079   
BEL         0.522680    0.585672    0.224594    0.156953  0.330512   
BHARTIARTL  0.282859    0.316537    0.236810    0.146953  0.238602   
BPCL        0.455289    0.495117    0.174973    0.215787  0.222936   
BRITANNIA   0.130655    0.133400    0.231736    0.42

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import urllib.parse

DB_USER = "postgres"
DB_PASSWORD = urllib.parse.quote_plus("Mahivalli@24")
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "stockhost"
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

df = pd.read_csv("master_csv.csv")
pivot_df = df.pivot(index="date", columns="Ticker", values="close")
returns_df = pivot_df.pct_change().dropna()

corr_matrix = returns_df.corr()
print("Correlation Matrix of Stock Returns:")
print(corr_matrix)

corr_matrix_reset = corr_matrix.reset_index().rename(columns={"index": "Ticker"})
corr_matrix_reset.to_sql("correlation_matrix", engine, if_exists="replace", index=False)
print("✅ Correlation matrix stored in DB (correlation_matrix)")

corr_pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .rename_axis(["Ticker1", "Ticker2"])
    .reset_index(name="Correlation")
)

top5_corr = corr_pairs.sort_values("Correlation", ascending=False).head(5)
print("\nTop 5 Highly Correlated Ticker Pairs:")
print(top5_corr)

top5_corr.to_sql("top5_correlated_pairs", engine, if_exists="replace", index=False)
print("✅ Top 5 correlated pairs stored in DB (top5_correlated_pairs)")


Correlation Matrix of Stock Returns:
Ticker      ADANIENT  ADANIPORTS  APOLLOHOSP  ASIANPAINT  AXISBANK  \
Ticker                                                               
ADANIENT    1.000000    0.874233    0.136501    0.291583  0.297736   
ADANIPORTS  0.874233    1.000000    0.168902    0.274045  0.380514   
APOLLOHOSP  0.136501    0.168902    1.000000    0.258055  0.179381   
ASIANPAINT  0.291583    0.274045    0.258055    1.000000  0.126055   
AXISBANK    0.297736    0.380514    0.179381    0.126055  1.000000   
BAJAJ-AUTO  0.201871    0.204456    0.223900    0.167650  0.198477   
BAJAJFINSV  0.356547    0.398041    0.249392    0.248090  0.393683   
BAJFINANCE  0.336031    0.398769    0.223142    0.296922  0.354079   
BEL         0.522680    0.585672    0.224594    0.156953  0.330512   
BHARTIARTL  0.282859    0.316537    0.236810    0.146953  0.238602   
BPCL        0.455289    0.495117    0.174973    0.215787  0.222936   
BRITANNIA   0.130655    0.133400    0.231736    0.426

In [12]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse

DB_USER = "postgres"
DB_PASSWORD = urllib.parse.quote_plus("Mahivalli@24")
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "stockhost"
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

df = pd.read_csv("master_csv.csv")
df["date"] = pd.to_datetime(df["date"])
df["YearMonth"] = df["date"].dt.to_period("M").astype(str)


monthly_returns = df.groupby(["Ticker", "YearMonth"])["close"].apply(
    lambda x: x.iloc[-1] / x.iloc[0] - 1 if len(x) > 1 else None
).reset_index(name="MonthlyReturn")

top_gainers = (
    monthly_returns.groupby("YearMonth")
    .apply(lambda x: x.nlargest(5, "MonthlyReturn"))
    .reset_index(drop=True)
)

top_losers = (
    monthly_returns.groupby("YearMonth")
    .apply(lambda x: x.nsmallest(5, "MonthlyReturn"))
    .reset_index(drop=True)
)

top_gainers.to_sql("monthly_top_gainers", engine, if_exists="replace", index=False)
top_losers.to_sql("monthly_top_losers", engine, if_exists="replace", index=False)

print(" Top 5 monthly gainers saved to DB (monthly_top_gainers)")
print(" Top 5 monthly losers saved to DB (monthly_top_losers)")

C:\Users\shenbagam\AppData\Local\Temp\ipykernel_18420\1631092241.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(5, "MonthlyReturn"))
C:\Users\shenbagam\AppData\Local\Temp\ipykernel_18420\1631092241.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nsmallest(5, "MonthlyReturn"))


 Top 5 monthly gainers saved to DB (monthly_top_gainers)
 Top 5 monthly losers saved to DB (monthly_top_losers)
